# Eigen y Python

En linux se puede instalar Eigen con el paquete libeigen3-dev. También se puede bajar la carpeta eigen para poder elegir bien la versión.

In [2]:
# !git clone https://gitlab.com/libeigen/eigen.git
!apt-get install libeigen3-dev

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


# Ejemplo Eigen básico

%%file file_name, copia todo el contendio de la celda y lo agrega al archivo file_name

In [2]:
%%file eigen_test.cpp


#include <iostream>
#include <eigen3/Eigen/Dense>

int main()
{
  Eigen::Matrix2d a;
  a << 1, 2,
       3, 4;
  Eigen::MatrixXd b(2,2);   // matriz de tamanio dinamico
  b << 2, 3,
       1, 4;
  std::cout << "a + b =\n" << a + b << std::endl;
  std::cout << "a - b =\n" << a - b << std::endl;
  std::cout << "Doing a += b;" << std::endl;
  a += b;
  std::cout << "Now a =\n" << a << std::endl;
  Eigen::Vector3d v(1,2,3);
  Eigen::Vector3d w(1,0,0);
  std::cout << "-v + w - v =\n" << -v + w - v << std::endl;
}

Overwriting eigen_test.cpp


In [3]:
%%bash

# g++ -I eigen eigen_test.cpp -o out
g++ eigen_test.cpp -o out
./out

a + b =
3 5
4 8
a - b =
-1 -1
 2  0
Doing a += b;
Now a =
3 5
4 8
-v + w - v =
-1
-4
-6


## Ejemplo con una función de producto matriz vector

In [6]:
%%file eigen_types_test.cpp

#include <iostream>
#include <eigen3/Eigen/Dense>

using Eigen::MatrixXd;
using Eigen::VectorXd;

VectorXd matrix_vector_multiplication(const MatrixXd& matrix, const VectorXd& vector) {
    return matrix * vector;
}


int main() {
    // Example usage
    MatrixXd A(3, 3);
    A << 1, 2, 3,
         4, 5, 6,
         7, 8, 9;

    VectorXd b(3);
    b << 1, 2, 3;

    VectorXd c = matrix_vector_multiplication(A, b);

    std::cout << "Result of matrix-vector multiplication: " << c.transpose() << std::endl;

    return 0;
}

Writing eigen_types_test.cpp


## Compilamos

In [7]:
%%bash
# g++ -Ieigen eigen_types_test.cpp -o out
g++ eigen_types_test.cpp -o out
./out

Result of matrix-vector multiplication: 14 32 50


## Comparamos con numpy

In [4]:
import numpy as np
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
b = np.array([1, 2, 3], dtype=np.float64)
A@b

array([14., 32., 50.])

# Opción con entrada y salida de archivos de texto

In [9]:
%%file eigen_types_iofile_test.cpp

#include <iostream>
#include <fstream>
#include <eigen3/Eigen/Dense>

using Eigen::MatrixXd;
using Eigen::VectorXd;

VectorXd matrix_vector_multiplication(const MatrixXd& matrix, const VectorXd& vector) {
    return matrix * vector;
}

int main(int argc, char** argv) {
    if (argc != 3) {
        std::cerr << "Usage: " << argv[0] << " input_file output_file" << std::endl;
        return 1;  // codigo 1 para cantidad incorrecta de argumentos.
    }

    const char* input_file = argv[1];
    const char* output_file = argv[2];

    std::ifstream fin(input_file);
    if (!fin.is_open()) {
        std::cerr << "Error: could not open input file " << input_file << std::endl;
        return 1;
    }

    // Read matrix and vector from file
    int nrows, ncols;
    fin >> nrows >> ncols;

    MatrixXd A(nrows, ncols);
    for (int i = 0; i < nrows; i++) {
        for (int j = 0; j < ncols; j++) {
            fin >> A(i, j);
        }
    }

    VectorXd b(ncols);
    for (int i = 0; i < ncols; i++) {
        fin >> b(i);
    }

    fin.close();

    // Perform matrix-vector multiplication
    VectorXd c = matrix_vector_multiplication(A, b);

    // Write result to output file
    std::ofstream fout(output_file);
    if (!fout.is_open()) {
        std::cerr << "Error: could not open output file " << output_file << std::endl;
        return 1;
    }

    fout << c.transpose() << std::endl;

    fout.close();

    return 0;
}


Writing eigen_types_iofile_test.cpp


## Escribimos un archivo de texto con numpy

!rm input_data.txt  # borra el archivo si es que ya existe.

with foo as bar :
crea un contexto en el que vamos a trabajar, y se encarga de manejar las excepciones... si lo usamos con archivos, va a cerrar el archivo automaticamente al terminar su scope, aunque se hayan producido exepciones.

In [5]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)
b = np.array([1, 2, 3], dtype=np.float64)

# borramos el archivo input si ya existe
!rm input_data.txt 

with open('input_data.txt','a') as fin: 
    fin.write(f"{A.shape[0]} {A.shape[1]}\n")
    np.savetxt(fin,A, newline="\n")

    # si no hacemos reshape a b,lo imprime como vector columna!
    # reshape(1,-1) es volverlo matriz de 1 row, y cuantas cols necesite (-1)
    np.savetxt(fin, b.reshape(1,-1), fmt='%1.3f', newline="\n")

!cat input_data.txt


3 3
1.000000000000000000e+00 2.000000000000000000e+00 3.000000000000000000e+00
4.000000000000000000e+00 5.000000000000000000e+00 6.000000000000000000e+00
7.000000000000000000e+00 8.000000000000000000e+00 9.000000000000000000e+00
1.000 2.000 3.000


## Compilamos y ejecutamos

In [17]:
%%bash

# compilamos
g++ eigen_types_iofile_test.cpp -o out

# llamamos a la funcion con sus argumentos por linea de comandos
./out input_data.txt output_data.txt

# printeamos el resultado del archivo output
cat output_data.txt

14 32 50


In [19]:
np.loadtxt('output_data.txt')

array([14., 32., 50.])

# Usando Python ctypes

Map<MatrixXd> es para crear un objeto que utiliza un pedazo de memoria existente!!
Osea, mapea una region de memoria para que sea una MatrixXd en este caso!
Los parametros que toman estos mapas, son un puntero a la region de memoria y las dimensiones
del objeto.

In [22]:
%%file eigen_ctypes_test.cpp

#include <iostream>
#include <eigen3/Eigen/Dense>
#include <dlfcn.h>

using namespace Eigen;

extern "C" {
    void matrix_vector_multiply(double* matrix, double* vector, double* result, int rows, int cols) {
        Map<MatrixXd> mat(matrix, rows, cols);
        Map<VectorXd> vec(vector, cols);
        Map<VectorXd> res(result, rows);
        res = mat * vec ;
    }
}

Overwriting eigen_ctypes_test.cpp


The .so file extension stands for "Shared Object". It is used for dynamically linked libraries in Unix-like operating systems, including Linux. These libraries are loaded at runtime by the operating system when they are required by a program. They are similar to DLL files in Windows.

In [23]:
!rm eigen_ctypes_test.so # borramos por las dudas

# compila el .cpp en una shared library .so  (shared object)
!g++ -shared -fPIC -Llibdl -o eigen_ctypes_test.so eigen_ctypes_test.cpp

!ls # chequeamos que este la lib .so

clase.pdf	       eigen_types_iofile_test.cpp  met_pot_alu_20241C.ipynb
eigen_ctypes_test.cpp  eigen_types_test.cpp	    out
eigen_ctypes_test.so   Eigen_y_Python.ipynb	    output_data.txt
eigen_test.cpp	       input_data.txt


The provided Python code defines a class sharedlib that is used to load and interact with shared libraries (also known as dynamic link libraries). This is done using the ctypes module, which provides C-compatible data types and allows calling functions in DLLs/shared libraries.

Here's a breakdown of the class:

- dlclose = ctypes.CDLL(None).dlclose: This line loads the dlclose function from the standard C library. dlclose is a function that closes a library previously opened by dlopen. Note that this won't work on Windows because dlclose is not available in the Windows C runtime.

- dlclose.argtypes = (ctypes.c_void_p,): This line sets the argument types for the dlclose function. It expects a single argument of type void* (pointer to void).

__init__(self, path, method, *args): This is the constructor of the sharedlib class. It takes a path to the shared library, the name of a method in the library, and a variable number of arguments that specify the argument types of the method.

- self.lib = ctypes.cdll.LoadLibrary(f'./{path}'): This line loads the shared library at the given path.

- self.method_object = getattr(self.lib, method): This line gets the method from the library.

- self.method_object.argtypes = args: This line sets the argument types for the method. This is necessary because ctypes needs to know the argument types to correctly convert Python values to C values.

__call__(self, *args): This method allows instances of the class to be called like functions. It calls the method in the shared library with the given arguments.

- unload(self): This method unloads the shared library by repeatedly calling dlclose until it fails (which indicates that the library has been successfully unloaded).

The last lines of the code create an instance of the sharedlib class for a shared library named eigen_ctypes_test.so and a method named matrix_vector_multiply. The method takes five arguments: three pointers to doubles and two integers.

In [26]:
import ctypes
import numpy as np

class sharedlib():
    # dlclose es una funcion en una variable    
    dlclose = ctypes.CDLL(None).dlclose  # This WON'T work on Win
    dlclose.argtypes = (ctypes.c_void_p,)

    def __init__(self, path, method, *args):
        
        # en el atributo lib, tenemos la libreria que importamos.
        self.lib = ctypes.cdll.LoadLibrary(f'./{path}')

        # en method_object tenemos el metodo que queremos usar de la libreria que importamos.
        self.method_object = getattr(self.lib, method)

        # Se explicitan los tipos de los argumentos para el método deseado
        self.method_object.argtypes = args

    # de esta manera, podemos llamar a una instancia de la clase como una funcion
    def __call__(self, *args):
        # al llamar a la instancia, ejecutamos el metodo de C que le cargamos en init
        return self.method_object(*args)

    def unload(self):
        while self.dlclose(self.lib._handle) != -1:
            pass


lib = sharedlib('eigen_ctypes_test.so', 'matrix_vector_multiply',
                                                                ctypes.POINTER(ctypes.c_double),
                                                                ctypes.POINTER(ctypes.c_double),
                                                                ctypes.POINTER(ctypes.c_double),
                                                                ctypes.c_int,
                                                                ctypes.c_int
                                                            )

# Creamos la matriz de entrada
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=np.float64)

# Eigen la mapea transpuesta a la información del puntero así que hay que pasarlo a orden tipo Fortran
A = np.asfortranarray(A)
b = np.array([1, 2, 3], dtype=np.float64)

# Definimos el vector donde se van a guardar los resultados
result = np.zeros((3,), dtype=np.float64)

# Llamamos a nuestra función en C++ pasando los argumentos de entrada
lib(
    A.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    b.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    result.ctypes.data_as(ctypes.POINTER(ctypes.c_double)),
    ctypes.c_int(A.shape[0]),
    ctypes.c_int(A.shape[1])
)

lib.unload() # para poder recompilar la lib hay que cerrarla

result

array([14., 32., 50.])

# Cython

In [30]:
!pip install cython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 15.3 MB/s eta 0:00:00m eta 0:00:010:01


In [6]:
%reload_ext cython

This Python code is written in Cython, a programming language that aims to be a superset of Python, designed to give C-like performance with code that is written mostly in Python. Cython is often used to optimize Python code and to extend Python's capabilities by interfacing with C libraries.

%%cython -a --verbose

This line is a magic command in Jupyter notebook that allows the cell to be run with Cython. The -a flag is used to produce an annotated version of the code, which shows how the Python code is translated into C. The --verbose flag is used to provide more detailed output about the compilation process.

- from libc.math cimport sin, cos

This line imports the sin and cos functions from the C standard library's math module. In Cython, cimport is used to import C functions, types, or variables.

- def r_cython(double[:] x_vec, double[:] y_vec):

This line defines a function r_cython that takes two 1D arrays of doubles as arguments. The double[:] syntax is a Cython type declaration for a memoryview, which is a data structure that provides efficient access to arrays of homogeneous C types.

- cdef double s = 0 and cdef int i

These lines declare two C variables, s of type double and i of type int. In Cython, cdef is used to declare C variables.

- for i in range(len(x_vec)): s += cos(x_vec[i])*sin(y_vec[i])

This loop iterates over each element in x_vec and y_vec, computes the cosine of the element in x_vec times the sine of the element in y_vec, and adds the result to s.

- return s

This line returns the final value of s.

In [7]:
%%cython -a --verbose

import math
# use C math functions
from libc.math cimport sin, cos

# use C types instead of Python types
def r_cython(double[:] x_vec, double[:] y_vec):
    cdef double s = 0
    cdef int i
    for i in range(len(x_vec)):
        s += cos(x_vec[i])*sin(y_vec[i])
    return s

[1/1] Cythonizing /home/pablo/.cache/ipython/cython/_cython_magic_24ab382f3e71fd51fa3da52eb6cfeb6826dfa5cd.pyx
building '_cython_magic_24ab382f3e71fd51fa3da52eb6cfeb6826dfa5cd' extension
x86_64-linux-gnu-gcc -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/include/python3.10 -c /home/pablo/.cache/ipython/cython/_cython_magic_24ab382f3e71fd51fa3da52eb6cfeb6826dfa5cd.c -o /home/pablo/.cache/ipython/cython/home/pablo/.cache/ipython/cython/_cython_magic_24ab382f3e71fd51fa3da52eb6cfeb6826dfa5cd.o
x86_64-linux-gnu-gcc -shared -Wl,-O1 -Wl,-Bsymbolic-functions -Wl,-Bsymbolic-functions -g -fwrapv -O2 -Wl,-Bsymbolic-functions -g -fwrapv -O2 -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 /home/pablo/.cache/ipython/cython/home/pablo/.cache/ipython/cython/_cyt

In [8]:
x = np.ones(10)
r_cython(x, x)

4.546487134128409

In [34]:
!python -m pip install eigency
%reload_ext cython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.4 MB/s eta 0:00:00m eta 0:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for eigency: filename=eigency-3.4.0.2-cp311-cp311-linux_x86_64.whl size=1651106 sha256=ee38a48ba484bb9031ee0ffc5c8de9359b9dc2abcec5acf889b580a2f4ffe356
  Stored in directory: /home/pablo/.cache/pip/wheels/02/fd/6d/30884c99fdf164a8add4dca4e908521a2d8840bb9673beca47
Successfully built eigency


In [41]:
%%writefile functions.h
Eigen::MatrixXd function_w_mat_arg(const Eigen::Map<Eigen::MatrixXd> &mat) {
    std::cout << mat << "\n";
    return mat;
}

Overwriting functions.h


In [39]:
import numpy
print(numpy.get_include())

/home/pablo/anaconda3/lib/python3.11/site-packages/numpy/core/include


In [9]:
%reload_ext cython

# los magic commands con %% o % deben estar en la primera linea siempre

This Python code is written in Cython, a programming language that is a superset of Python designed to give C-like performance with code that is written mostly in Python. This code is specifically designed to interface with C++ code, using the Eigen library for linear algebra and the numpy library for handling arrays.

%%cython -a --verbose -+ -I /home/pablo/anaconda3/lib/python3.11/site-packages/numpy/core/include/ -I /usr/include/eigen3 -I ./

This line is a magic command in Jupyter notebook that allows the cell to be run with Cython. The -a flag is used to produce an annotated version of the code, which shows how the Python code is translated into C. The --verbose flag is used to provide more detailed output about the compilation process. The -+ flag enables C++ mode. The -I flags are used to specify the include paths for the numpy and Eigen libraries, as well as the current directory.

- import math

This line imports the Python math module. However, it's not used in the code and could be removed.

- from libc.math cimport sin, cos

This line imports the sin and cos functions from the C standard library's math module. In Cython, cimport is used to import C functions, types, or variables.

- from eigency.core cimport *

This line imports all the functions and types from the eigency.core module, which is a Cython interface to the Eigen library.

- cdef extern from "functions.h": cdef MatrixXd _function_w_mat_arg "function_w_mat_arg"(Map[MatrixXd] &)

These lines declare an external C++ function from the header file "functions.h". The function is named _function_w_mat_arg in Cython and function_w_mat_arg in C++. It takes a reference to a mapped Eigen MatrixXd object as an argument.

- def function_w_mat_arg(np.ndarray array): return ndarray(_function_w_mat_arg(Map[MatrixXd](array)))

These lines define a Python function that wraps the C++ function. It takes a numpy array as an argument, maps it to an Eigen MatrixXd object, passes it to the C++ function, and then converts the result back to a numpy array.

This code is an example of how Cython can be used to interface with C++ code and libraries, providing a way to use powerful C++ libraries like Eigen in Python code.

In [10]:
%%cython -a --verbose -+ -I /home/pablo/anaconda3/lib/python3.11/site-packages/numpy/core/include/ -I /usr/include/eigen3 -I ./

import math
# use C math functions
from libc.math cimport sin, cos
# importamos las cosas de eigen mediante eigency, una interfaz de cython
from eigency.core cimport *

cdef extern from "functions.h":
     # declara la funcion _function_... en cyton, a partir de function_w... en cpp
     cdef MatrixXd _function_w_mat_arg "function_w_mat_arg"(Map[MatrixXd] &)

# use C types instead of Python types
# definimos funcion wrapper a la de C
def function_w_mat_arg(np.ndarray array):
    return ndarray(_function_w_mat_arg(Map[MatrixXd](array)))

[1/1] Cythonizing /home/pablo/.cache/ipython/cython/_cython_magic_6419ee7e652e3ed1ad07d1c517e9da7269338b46.pyx



Error compiling Cython file:
------------------------------------------------------------
...

import math
# use C math functions
from libc.math cimport sin, cos
# importamos las cosas de eigen mediante eigency, una interfaz de cython
from eigency.core cimport *
^
------------------------------------------------------------

/home/pablo/.cache/ipython/cython/_cython_magic_6419ee7e652e3ed1ad07d1c517e9da7269338b46.pyx:6:0: 'eigency/core.pxd' not found

Error compiling Cython file:
------------------------------------------------------------
...
# importamos las cosas de eigen mediante eigency, una interfaz de cython
from eigency.core cimport *

cdef extern from "functions.h":
     # declara la funcion _function_... en cyton, a partir de function_w... en cpp
     cdef MatrixXd _function_w_mat_arg "function_w_mat_arg"(Map[MatrixXd] &)
          ^
------------------------------------------------------------

/home/pablo/.cache/ipython/cython/_cython_magic_6419ee7e652e3ed1ad07d1c517e9da7269

In [10]:
function_w_mat_arg(np.ones((3,3)))

# lo primero que printea, es lo de la funcion de C

1 1 1
1 1 1
1 1 1


array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

# pybind11

In [11]:
get_ipython()

In [11]:
if "google.colab" in str(get_ipython()):
    !pip install git+https://github.com/aldanor/ipybind.git -qqq
%load_ext ipybind

ModuleNotFoundError: No module named 'ipybind'

In [12]:
%load_ext ipybind

This code is written in C++ and uses the Pybind11 library to create a Python extension module. Pybind11 is a lightweight header-only library that exposes C++ types in Python and vice versa.

- %%pybind11

This line is a magic command in Jupyter notebook that allows the cell to be run with Pybind11.

- #include <pybind11/numpy.h> #include <math.h> #include <eigen3/Eigen/Dense> #include <iostream>

These lines include necessary header files. pybind11/numpy.h is for Pybind11's numpy support, math.h is for C++ math functions, eigen3/Eigen/Dense is for the Eigen library which provides high-level interfaces for linear algebra operations, and iostream is for input/output stream.

- PYBIND11_PLUGIN(example) { ... }

This macro creates a function that will be called when an import statement is issued from Python. The argument to the macro, example, is the name of the module.

- py::module m("example");

This line creates a new Python module named "example".

- m.def("r_pybind", [](const py::array_t<double>& x, const py::array_t<double>& y) { ... });

This line adds a function named r_pybind to the module. The function is defined using a C++ lambda expression. It takes two 1D numpy arrays of doubles as arguments.

- double sum{0}; auto rx{x.unchecked<1>()}; auto ry{y.unchecked<1>()};

These lines declare a double variable sum and two py::array_t<double> variables rx and ry which are unchecked copies of the input arrays. Unchecked access is faster but does not perform bounds checking or broadcasting.

- for (py::ssize_t i = 0; i < rx.shape(0); i++){ sum += std::cos(rx[i])*std::sin(ry[i]); }

This loop iterates over each element in rx and ry, computes the cosine of the element in rx times the sine of the element in ry, and adds the result to sum.

- Eigen::Matrix2d a; a << 1, 2, 3, 4;

These lines declare a 2x2 Eigen matrix and assign values to it. However, this matrix is not used in the code and could be removed.

- return sum;

This line returns the final value of sum.

- return m.ptr();

This line returns a pointer to the Python module.

This code is an example of how Pybind11 can be used to create Python extension modules in C++, allowing for efficient numerical computations and easy integration with C++ libraries like Eigen.

In [13]:
%%pybind11

#include <pybind11/numpy.h>
#include <math.h>
#include <eigen3/Eigen/Dense>
#include <iostream>
PYBIND11_PLUGIN(example) {
    py::module m("example");
    m.def("r_pybind", [](const py::array_t<double>& x, const py::array_t<double>& y) {
        double sum{0};
        auto rx{x.unchecked<1>()};
        auto ry{y.unchecked<1>()};
        for (py::ssize_t i = 0; i < rx.shape(0); i++){
            sum += std::cos(rx[i])*std::sin(ry[i]);
        }
        Eigen::Matrix2d a;
        a << 1, 2,
            3, 4;

        return sum;

    });
    return m.ptr();
}

/home/pablo/.cache/ipython/pybind11/pybind11_d41c85d.cpp: In function ‘PyObject* pybind11_init()’:
/home/pablo/.cache/ipython/pybind11/pybind11_d41c85d.cpp:8:27: warning: ‘pybind11::module_::module_(const char*, const char*)’ is deprecated: Use PYBIND11_MODULE or module_::create_extension_module instead [-Wdeprecated-declarations]
    8 |     py::module m("example");
      |                           ^
In file included from /home/pablo/.local/lib/python3.10/site-packages/ipybind/include/pybind11_preamble.h:1,
                 from /home/pablo/.cache/ipython/pybind11/pybind11_d41c85d.cpp:1:
/home/pablo/.local/lib/python3.10/site-packages/pybind11/include/pybind11/pybind11.h:1209:14: note: declared here
 1209 |     explicit module_(const char *name, const char *doc = nullptr) {
      |              ^~~~~~~
lto-wrapper: warning: using serial compilation of 2 LTRANS jobs


In [14]:
r_pybind(x, x)

4.546487134128409